In [ ]:
import warnings
warnings.filterwarnings("ignore")
import os

#To disable all logging output from TensorFlow
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

import numpy as np
import pandas as pd 
from matplotlib import pyplot as plt
import tensorflow as tf

from tensorflow.keras import backend as K
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Input
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import LabelEncoder

In [ ]:
df=pd.read_csv("transformed_df.csv")

In [ ]:
def replace_with_unique(df, column: str):
    uniques = df[column].unique()
    mapping = dict(zip(uniques, range(0, len(uniques))))
    df[column].replace(mapping, inplace=True)
    return df
replace_with_unique(df, "diagnostic")


In [ ]:
df_naked=df[['region_ABDOMEN', 'region_ARM', 'region_BACK', 'region_CHEST',
       'region_EAR', 'region_FACE', 'region_FOOT', 'region_FOREARM',
       'region_HAND', 'region_LIP', 'region_NECK', 'region_NOSE',
       'region_SCALP', 'region_THIGH', 'age', 'bleed', 'grew', 'itch',
       'elevation', 'changed', 'smoke', 'pesticide', 'hurt', 'diagnostic',]]
df_naked.columns

In [ ]:
import numpy as np
X  = np.array(df.drop(df[["diagnostic","img_id", "image_path"]], axis=1))
y = np.array(df["diagnostic"])

In [ ]:
X.shape

In [ ]:
y.shape

In [ ]:
from tensorflow.keras.utils import to_categorical
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)
y = to_categorical(y)

In [ ]:
K.clear_session()
model = Sequential()

In [ ]:
model = Sequential()
model.add(Input(shape=X.shape[1]))
model.add(Dense(units=32, activation='relu'))
model.add(Dense(units=16, activation='relu'))
model.add(Dense(units=8, activation='relu'))
model.add(Dense(5,activation='softmax'))
model.summary()

In [30]:
from tensorflow import keras
model.compile(optimizer=Adam(), loss=keras.losses.categorical_crossentropy, metrics=[keras.metrics.categorical_accuracy])
history = model.fit(X, y, epochs=10_000, validation_split=0.2)

In [ ]:
history.history.keys()

In [ ]:
pd.DataFrame(history.history).plot()

In [ ]:
model.save("model_moons.h5")

In [ ]:
ypred = model.predict(X)
plt.scatter(X[:, 0], X[:, 1], c=ypred[:, 0])